필요 라이브러리 설치

In [ ]:
!pip install trimesh matplotlib
!pip install manifold3d

import trimesh, zipfile,glob, io
import matplotlib.pyplot as plt
from google.colab import files
import numpy as np
from scipy.ndimage import gaussian_filter
from PIL import Image, ImageOps # PIL 추가

STL 파일 분석

In [ ]:
uploaded = files.upload()
stl_filename = list(uploaded.keys())[0]

mesh = trimesh.load(io.BytesIO(uploaded[stl_filename]), file_type='stl')

# Z 기준 슬라이싱
z_range = mesh.bounds[1, 2] - mesh.bounds[0, 2]
num_slices = int(z_range)
slice_thickness = 0.01

slices = []
for z in np.linspace(mesh.bounds[0, 2], mesh.bounds[1, 2], num_slices):
    plane_origin = [0, 0, z]
    plane_normal = [0, 0, 1]
    plane = trimesh.creation.box(extents=[100, 100, slice_thickness], transform=trimesh.transformations.translation_matrix(plane_origin))
    slice = trimesh.boolean.intersection([mesh, plane])
    slices.append(slice)

for i, slice in enumerate(slices):
    if slice is not None:
        vertices = np.array(slice.vertices)
        min_x, min_y = np.min(vertices[:, :2], axis=0)
        max_x, max_y = np.max(vertices[:, :2], axis=0)

        max_size = 500
        width = min(int(max_x - min_x) + 1, max_size)
        height = min(int(max_y - min_y) + 1, max_size)
        image = np.zeros((height, width))

        # sigma 값 조절 (예: 2.0)
        image = gaussian_filter(image, sigma=2.0)

        for vertex in vertices:
            x, y = int(vertex[0] - min_x), int(vertex[1] - min_y)
            if 0 <= x < width and 0 <= y < height:
                image[y, x] = 1

        fig, ax = plt.subplots()
        ax.imshow(image, cmap='gray')
        ax.axis('off')
        plt.savefig(f'slice_{i}.png')
        plt.close(fig)

        # 보정
        img = Image.open(f'slice_{i}.png').convert("RGB")
        img = ImageOps.expand(img, border=10, fill='white')
        img.save(f'slice_{i}.png')

    else:
        print(f"Slice {i} is empty.")

# ZIP 파일 생성 및 다운로드
with zipfile.ZipFile('model_slices.zip', 'w') as zipf:
    for filename in glob.glob('slice_*.png'):
        zipf.write(filename)

files.download('model_slices.zip')

모서리 다듬기 (오류 있음)

In [ ]:
from scipy.ndimage import gaussian_filter
        image = gaussian_filter(image, sigma=0.5)

        fig, ax = plt.subplots()
        ax.imshow(image, cmap='gray')
        ax.axis('off')
        plt.savefig(f'slice_{i}.png')
        plt.close(fig)
    else:
        print(f"Slice {i} is empty.")